In [1]:
# general imports
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# TARDIS imports for the gamma-ray code
from tardis.io.atom_data import AtomData
from tardis.model import SimulationState
from tardis.plasma.base import BasePlasma
from tardis.io.configuration import config_reader
from tardis.io.atom_data.util import download_atom_data
from tardis.model.matter.decay import IsotopicMassFraction
from tardis.energy_input.energy_source import get_nuclear_lines_database
from tardis.energy_input.gamma_ray_channel import create_isotope_dicts, create_inventories_dict, calculate_total_decays, create_isotope_decay_df, \
    distribute_packets, sample_single_packet, sample_packets

from tardis.plasma.properties import Density, Abundance, IsotopeAbundance, \
    IsotopeNumberDensity, AtomicData, AtomicMass, IsotopeMass, NumberDensity, \
    SelectedAtoms

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
# Download the atom data file from tardis-refdata repo to run this cell.
download_atom_data('kurucz_cd23_chianti_H_He')
atom_data_file = 'kurucz_cd23_chianti_H_He.h5'
atom_data = AtomData.from_hdf(atom_data_file)

INFO:tardis.io.atom_data.base:Reading Atom Data with: UUID = 6f7b09e887a311e7a06b246e96350010 MD5  = 864f1753714343c41f99cb065710cace 
INFO:tardis.io.atom_data.base:Non provided Atomic Data: synpp_refs, photoionization_data, yg_data, two_photon_data, linelist


In [3]:
# Read the config file and create a configuration object
config = config_reader.Configuration.from_yaml("tardis_configv1_density_exponential_nebular.yml")

In [4]:
# Create the model
model = SimulationState.from_csvy(config, atom_data)


INFO:tardis.model.matter.decay:Decaying abundances for 2600540.0352 seconds
/Users/anirbandutta/Software/tardis/tardis/model/matter/composition.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '7.961942738176252e-23' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  isotope_mass_df[isotope_tuple] = isotope_mass


In [5]:
gamma_ray_lines = get_nuclear_lines_database(atom_data_file)

In [6]:
raw_isotope_abundance = model.composition.raw_isotope_abundance

In [19]:
t_steps = 50
time_range = np.linspace(0.1, 10, t_steps)

abundance = [IsotopicMassFraction(raw_isotope_abundance).decay(delta_t) for delta_t in time_range]

INFO:tardis.model.matter.decay:Decaying abundances for 8640.0 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 26096.326530612245 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 43552.6530612245 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 61008.97959183674 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 78465.30612244898 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 95921.63265306124 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 113377.95918367349 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 130834.28571428572 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 148290.61224489796 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 165746.9387755102 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 183203.26530612248 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 200659.59183673473 seconds
INFO:tardis.model.matter.decay:Dec

In [20]:
abundance

[                                     0             1             2   \
 atomic_number mass_number                                             
 22            48           8.018787e-17  9.086719e-17  9.130005e-17   
 23            48           3.646544e-14  4.132186e-14  4.151870e-14   
 24            48           4.556181e-13  5.162968e-13  5.187563e-13   
               52           5.878567e-17  8.179860e-17  9.204355e-17   
 25            52           1.469548e-17  2.044835e-17  2.300942e-17   
 26            52           3.347253e-16  4.657608e-16  5.240955e-16   
               56           1.028476e-24  1.814353e-24  2.454927e-24   
 27            56           2.287349e-21  4.035153e-21  5.459803e-21   
 28            56           1.994280e-19  3.518146e-19  4.760262e-19   
 
                                      3             4             5   \
 atomic_number mass_number                                             
 22            48           3.228974e-09  2.616120e-09  2.4671

In [8]:
# Construct the Plasma

input = [Density, Abundance, IsotopeAbundance, AtomicData, AtomicMass, IsotopeNumberDensity, NumberDensity, SelectedAtoms, IsotopeMass]

plasma = BasePlasma(plasma_properties=input, density = model.density, 
                    abundance=model.abundance, isotope_abundance=model.composition.raw_isotope_abundance,
                    atomic_data = atom_data)

In [9]:
shell_masses = model.volume * model.density

In [10]:
isotope_dict = create_isotope_dicts(raw_isotope_abundance, shell_masses)

In [11]:
inventories_dict = create_inventories_dict(isotope_dict)

In [12]:
total_decays = calculate_total_decays(inventories_dict, 80)

In [13]:
isotope_decay_df = create_isotope_decay_df(total_decays, gamma_ray_lines)

In [14]:
isotope_decay_df = distribute_packets(isotope_decay_df)

In [17]:
packet = sample_single_packet(isotope_decay_df)

In [ ]:
sample_packets(isotope_decay_df, 1000)

In [18]:
type(packet)

pandas.core.frame.DataFrame